# Calculate effective temperature for Eelde/Groningen

## Libraries

In [1]:
from pandas import DataFrame, read_csv
import datetime
import math
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import pylab
from scipy.optimize import curve_fit
import os

## Set system parameters
### Working directory

In [2]:
working_directory ='/Users/peter/python/pyBuurtwarmte'
os.chdir(working_directory)

## Read KNMI data from Eelde airport


### KNMI format

```
# BRON: KONINKLIJK NEDERLANDS METEOROLOGISCH INSTITUUT (KNMI)
# Opmerking: door stationsverplaatsingen en veranderingen in waarneemmethodieken zijn deze tijdreeksen van dagwaarden mogelijk inhomogeen! Dat betekent dat deze reeks van gemeten waarden niet geschikt is voor trendanalyse. Voor studies naar klimaatverandering verwijzen we naar de gehomogeniseerde reeks maandtemperaturen van De Bilt <http://www.knmi.nl/kennis-en-datacentrum/achtergrond/gehomogeniseerde-reeks-maandtemperaturen-de-bilt> of de Centraal Nederland Temperatuur <http://www.knmi.nl/kennis-en-datacentrum/achtergrond/centraal-nederland-temperatuur-cnt>.
# 
# 
# STN      LON(east)   LAT(north)     ALT(m)  NAME
# 280:         6.585       53.125       5.20  EELDE
# 
# YYYYMMDD = Date (YYYY=year MM=month DD=day); 
# FG       = Daily mean windspeed (in 0.1 m/s); 
# TG       = Daily mean temperature in (0.1 degrees Celsius); 
# Q        = Global radiation (in J/cm2); 
# 
# STN,YYYYMMDD,   FG,   TG,    Q
# 
  280,20120101,   63,  108,  110
```
#### Clean and scrub data
1. skip first 12 rows
1. remove extra whitespaces from strings (because they remain in headings)
1. define YYYYMMDD data as string
1. re-index data frame - needs to be done before....
1. drop the empty data with NaN or NaT

In [3]:
# to delete a row that contans NaN or NaT check if dataframe has NaN and NaT as strings 
# which .dropna, .notnull and co. won't consider falsey:
# replace strings with "real" NaN, and then drop rows
# note that you have to use inplace = True if you want to make your dropna permanent

In [4]:
file = "KNMI_1965_2018.txt"
KNMI_Teff = pd.read_csv(file, 
                        skiprows=12, 
                        skipinitialspace=True, 
                        dtype={'YYYYMMDD': str})
KNMI_Teff.reset_index(drop=True)
KNMI_Teff.replace(["NaN", 'NaT'], np.nan, inplace = True)
KNMI_Teff.dropna(how='any', inplace = True)

#KNMI_Teff.reset_index(drop=True)

In [5]:
#KNMI_Teff

## Prepare data
1. add new column 'Date' and convert string date to machine readable datetime
1. make Date new index (N.B.: required for fast search of dates with .loc)

In [6]:
#KNMI_Teff['YYYYMMDD']

In [7]:
# add column 'Date' for machine readable date and set and re-index
KNMI_Teff['Date'] = KNMI_Teff['YYYYMMDD'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
KNMI_Teff.set_index('Date', inplace=True)
KNMI_Teff.reset_index(drop=True)

,# STN,YYYYMMDD,FG,TG,Q
0,280,19650101,67.0,34.0,130.0
1,280,19650102,31.0,6.0,318.0
2,280,19650103,26.0,-2.0,344.0
3,280,19650104,21.0,-1.0,397.0
4,280,19650105,51.0,32.0,163.0
5,280,19650106,51.0,54.0,45.0
6,280,19650107,46.0,52.0,72.0
7,280,19650108,62.0,62.0,63.0
8,280,19650109,67.0,44.0,203.0
9,280,19650110,46.0,41.0,210.0


In [8]:
pd.date_range(start = '1965-01-01', end = '2018-12-31' ).difference(KNMI_Teff.index)

DatetimeIndex(['1989-02-10', '1989-02-11', '1989-02-12', '1989-02-13',
               '1989-02-14', '1989-02-15', '1989-02-16', '1989-02-17',
               '1990-12-03', '1990-12-04', '1990-12-05', '1990-12-06',
               '1990-12-07', '1990-12-08', '1990-12-09', '1990-12-10',
               '1990-12-11'],
              dtype='datetime64[ns]', freq=None)

## Calculate effective temperature
The effective average day temperature is calculated as:
$T_{eff, day} = \overline{T}_{day} - C_u \cdot \overline{u}_{wind, day} + C_I \cdot \overline{I}_{rad, day} $

Where $\overline{T}_{day}$ is given in $^\circ C$, $\overline{u}_{wind, day}$ is given in $\frac{m}{s}$, and $\overline{I}_{rad, day}$ is given in $\frac{J}{cm^2}$.
      
The constants $C_u$ and $C_I$ have the following values and dimsions: $C_u = \frac{2}{3} (\frac{^\circ C}{\frac{m}{s}})$ and $C_I = \frac{1}{40 \cdot 24} (\frac{^\circ C}{\frac{J}{cm^2}})$

In [9]:
KNMI_Teff['Teff'] = KNMI_Teff['TG']/10 - (2/3 * (KNMI_Teff['FG'] / 10)) + (1 / (40 * 24)) * KNMI_Teff['Q']

## Calculate home sensitive effective temperature
The home sensitive effective temperature takes the last 3 days before the actual date of the effecive temperature into account. In essence the "composite" effective temperature is a weighted average of current and past $T_{eff}$ such that
$T_{eff, composite} = 0.53 \cdot T_{eff, day-0} + 0.27 \cdot T_{eff, day-1} + 0.13 \cdot T_{eff, day-2} + 0.07 \cdot T_{eff, day-3}$ 

add another culumn to the data frame and fill it with effective temperature

In [10]:
KNMI_Teff['Teff_lag'] = KNMI_Teff['Teff']

In [11]:
#start_date = pd.to_datetime('2012-01-01')+ pd.DateOffset(days= 3)

for i in KNMI_Teff.index:
        Teff_lag = KNMI_Teff.loc[i, 'Teff']
        date_min_3  = (i + pd.DateOffset(days= -3))
        date_min_2  = (i + pd.DateOffset(days= -2))
        date_min_1  = (i + pd.DateOffset(days= -1))
        date_min_0  = (i + pd.DateOffset(days= +0))
#        print(i)
        if ((date_min_3 in KNMI_Teff.index) and (date_min_2 in KNMI_Teff.index) and (date_min_1 in KNMI_Teff.index) and (date_min_0 in KNMI_Teff.index)):
            Teff_lag       = 0.53 * KNMI_Teff.at[date_min_0,'Teff'] \
                           + 0.27 * KNMI_Teff.at[date_min_1,'Teff'] \
                           + 0.13 * KNMI_Teff.at[date_min_2,'Teff'] \
                           + 0.07 * KNMI_Teff.at[date_min_3,'Teff']
#            print(i, date_min_3, date_min_2, date_min_1, date_min_0)
#            print(0.53 * KNMI_Teff.at[date_min_0,'Teff'] \
#                           + 0.27 * KNMI_Teff.at[date_min_1,'Teff'] \
#                           + 0.13 * KNMI_Teff.at[date_min_2,'Teff'] \
#                           + 0.07 * KNMI_Teff.at[date_min_3,'Teff'] , Teff_lag)
            KNMI_Teff.loc[i,'Teff_lag'] = Teff_lag
        else:
            print("not enough past data for the following  date: ", i)

not enough past data for the following  date:  1965-01-01 00:00:00
not enough past data for the following  date:  1965-01-02 00:00:00
not enough past data for the following  date:  1965-01-03 00:00:00
not enough past data for the following  date:  1989-02-18 00:00:00
not enough past data for the following  date:  1989-02-19 00:00:00
not enough past data for the following  date:  1989-02-20 00:00:00
not enough past data for the following  date:  1990-12-12 00:00:00
not enough past data for the following  date:  1990-12-13 00:00:00
not enough past data for the following  date:  1990-12-14 00:00:00


In [12]:
#to check specific date, fill in date and uncomment in next line
#KNMI_Teff.loc['2017-12-02']
# to check data uncomment next line
#KNMI_Teff

In [13]:
from bokeh.models import ColumnDataSource, HoverTool, Range1d, LinearAxis
from bokeh.plotting import figure, show, output_file
from bokeh.io import show, output_notebook, show

output_notebook()

source = ColumnDataSource(KNMI_Teff)

fig = figure()

fig  = figure(x_axis_type="datetime")
# Define x-axis
#fig.xaxis.axis_type = "datetime"
fig.xaxis.axis_label = 'Date'

# Define 1st LHS y-axis
fig.yaxis.axis_label = 'Temperature [˚C]'
fig.y_range = Range1d(start=-15, end=30)

# Create 1st RHS y-axis
fig.extra_y_ranges['temp'] = Range1d(start=-15, end=30)
fig.add_layout(LinearAxis(y_range_name='temp', axis_label='Temperature [°C]'), 'right')

fig.line(
    'Date',
    'Teff',
    source = source,
    legend = 'Temperature eff',
    line_width = 0.5,
    color = 'grey'
)

fig.line(
    'Date',
    'Teff_lag',
    source = source,
    legend = 'Temperature cum',
    y_range_name = 'temp',
    line_width = 0.2,
    color = 'black'
)

fig.add_tools(HoverTool(tooltips=[("Teff", "@Teff"), ("Teff_lag", "@Teff_lag")]))
fig.toolbar_location = 'above'

show(fig)

Loading BokehJS ...

## Write KNMI_Teff results to CSV file

In [14]:
file = "KNMI_Teff_1965_2018.csv"
export_report = KNMI_Teff.to_csv(file)

In [15]:
KNMI_Teff

,# STN,YYYYMMDD,FG,TG,Q,Teff,Teff_lag
Date,,,,,,,
1965-01-01,280,19650101,67.0,34.0,130.0,-0.931250,-0.931250
1965-01-02,280,19650102,31.0,6.0,318.0,-1.135417,-1.135417
1965-01-03,280,19650103,26.0,-2.0,344.0,-1.575000,-1.575000
1965-01-04,280,19650104,21.0,-1.0,397.0,-1.086458,-1.213865
1965-01-05,280,19650105,51.0,32.0,163.0,-0.030208,-0.593583
1965-01-06,280,19650106,51.0,54.0,45.0,2.046875,0.825198
1965-01-07,280,19650107,46.0,52.0,72.0,2.208333,1.643094
1965-01-08,280,19650108,62.0,62.0,63.0,2.132292,1.990344
1965-01-09,280,19650109,67.0,44.0,203.0,0.144792,1.082823
